In [1]:
import json
import spacy
import re
import difflib as dl
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
file_path = '/content/drive/MyDrive/NLP/gg2013.json'

#source1=https://catriscode.com/2021/03/02/extracting-or-removing-mentions-and-hashtags-in-tweets-using-python/
#source2== https://stackoverflow.com/questions/38365389/compare-similarity-between-names/38365913

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
with open('/content/drive/MyDrive/NLP/award_names.txt') as f:
    award_names = f.readlines()
for i in range(len(award_names)):
  award_names[i] = award_names[i][:-1]
print(award_names)

['best screenplay - motion picture', 'best director - motion picture', 'best performance by an actress in a television series - comedy or musical', 'best foreign language film', 'best performance by an actor in a supporting role in a motion picture', 'best performance by an actress in a supporting role in a series, mini-series or motion picture made for television', 'best motion picture - comedy or musical', 'best performance by an actress in a motion picture - comedy or musical', 'best mini-series or motion picture made for television', 'best original score - motion picture', 'best performance by an actress in a television series - drama', 'best performance by an actress in a motion picture - drama', 'best performance by an actor in a motion picture - comedy or musical', 'best motion picture - drama', 'best performance by an actor in a supporting role in a series, mini-series or motion picture made for television', 'best performance by an actress in a supporting role in a motion pictu

In [3]:
stop_words = list(stopwords.words('english'))
stop_words.append(' - ')
cleaned_award_names= []
for award in award_names:
  cleaned_award = ''
  for word in award.split():
    if word not in stop_words and word != '-':
      cleaned_award+=word+' '
  cleaned_award_names.append(cleaned_award[:-1])

print(cleaned_award_names)

['best screenplay motion picture', 'best director motion picture', 'best performance actress television series comedy musical', 'best foreign language film', 'best performance actor supporting role motion picture', 'best performance actress supporting role series, mini-series motion picture made television', 'best motion picture comedy musical', 'best performance actress motion picture comedy musical', 'best mini-series motion picture made television', 'best original score motion picture', 'best performance actress television series drama', 'best performance actress motion picture drama', 'best performance actor motion picture comedy musical', 'best motion picture drama', 'best performance actor supporting role series, mini-series motion picture made television', 'best performance actress supporting role motion picture', 'best television series drama', 'best performance actor mini-series motion picture made television', 'best performance actress mini-series motion picture made televisi

In [4]:
f = open(file_path)
data = json.load(f)
nlp = spacy.load("en_core_web_sm")
tweet_texts = []

for tweet in data:
  cleaned_tweet = re.sub('#([a-zA-Z0-9_]{1,50})', '', tweet['text'])
  cleaned_tweet = re.sub('@([a-zA-Z0-9_]{1,50})', '', cleaned_tweet)
  #cleaned_tweet = cleaned_tweet.lower()
  if 'rt @' not in cleaned_tweet:
    tweet_texts.append(cleaned_tweet)

In [5]:
def check_keyword(tweet_text,award_words):
  
  for award_word in award_words.split():
    if award_word not in tweet_text and award_word != '-':
      return False
  return True

print(check_keyword('nihaal subhash narayanan','nihaal narayanan'))

True


In [6]:
presenter_key_words = 'present|intro|announce|gave|presenters|presenter|presenting'

for award in cleaned_award_names[:]:
  hosts = {}
  for tweet_text in tweet_texts[:]:
    presenter_key_words_index = re.search(presenter_key_words,tweet_text.lower())
    #print(tweet_text)
    #print(award)
    if presenter_key_words_index and check_keyword(tweet_text.lower(),award.lower()):
      doc = nlp(tweet_text[:presenter_key_words_index.start()])
      for entity in doc.ents:
        if entity.label_ == 'PERSON':
          #print(tweet_text)
          #print()
          if entity.text in hosts.keys():
            hosts[entity.text] = hosts[entity.text] + 1
          else:
            hosts[entity.text] = 1

  hosts = dict(sorted(hosts.items(),key = lambda host:host[1],reverse=True))
  if (len(hosts.keys())!=0):
    print(award,end= ' : ')
    print(list(hosts.keys())[:2])
    print()

best screenplay motion picture : ['Robert Pattinson', 'Amanda Seyfried']

best foreign language film : ['Sylvester Stallone', 'Arnold Schwarzenegger']

best motion picture comedy musical : ['Dustin Hoffman']

best original score motion picture : ['Jason Statham']

best motion picture drama : ['Bill Clinton', 'Julia Roberts']

best animated feature film : ['Cohen']

